<a href="https://colab.research.google.com/github/rajgupt/dl-notebooks/blob/main/basic_ann.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [2]:
csv_file = tf.keras.utils.get_file('heart.csv', 'https://storage.googleapis.com/download.tensorflow.org/data/heart.csv')

16384/13273 [=====================================] - 0s 0us/step


In [3]:
# load dataset
df = pd.read_csv(csv_file)
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal,0


In [31]:
# create feature matrix using sklearn
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

In [66]:
target = df.pop('target')

In [67]:
vectorizer = ColumnTransformer([
                                ('norm', StandardScaler(), ['age', 'trestbps', 'chol', 'thalach', 'oldpeak', 'slope']),
                                ('ohe', OneHotEncoder(), ['sex', 'cp', 'fbs', 'restecg', 'exang', 'ca', 'thal'])
                                ])

In [68]:
X = vectorizer.fit_transform(df)
y = target.values

In [70]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [72]:
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(32)
val_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(32)

In [74]:
model = keras.Sequential()
model.add(keras.layers.Input(shape=(29,)))
model.add(keras.layers.Dense(10, activation=tf.nn.relu))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))
model.summary()

In [76]:
model.compile(optimizer='sgd', loss='binary_crossentropy', metrics='accuracy')

In [77]:
hist = model.fit(train_ds, epochs=100)

Epoch 1/100
8/8 [==============================] - 0s 3ms/step - loss: 0.6435 - accuracy: 0.6682
Epoch 2/100
8/8 [==============================] - 0s 2ms/step - loss: 0.6276 - accuracy: 0.6931
Epoch 3/100
8/8 [==============================] - 0s 2ms/step - loss: 0.6128 - accuracy: 0.7045
Epoch 4/100
8/8 [==============================] - 0s 2ms/step - loss: 0.5989 - accuracy: 0.7376
Epoch 5/100
8/8 [==============================] - 0s 3ms/step - loss: 0.5856 - accuracy: 0.7635
Epoch 6/100
8/8 [==============================] - 0s 3ms/step - loss: 0.5729 - accuracy: 0.7787
Epoch 7/100
8/8 [==============================] - 0s 3ms/step - loss: 0.5608 - accuracy: 0.7856
Epoch 8/100
8/8 [==============================] - 0s 3ms/step - loss: 0.5494 - accuracy: 0.7892
Epoch 9/100
8/8 [==============================] - 0s 3ms/step - loss: 0.5386 - accuracy: 0.7892
Epoch 10/100
8/8 [==============================] - 0s 4ms/step - loss: 0.5283 - accuracy: 0.7866
Epoch 11/100
8/8 [===========

In [78]:
model.evaluate(val_ds)

2/2 [==============================] - 0s 6ms/step - loss: 0.3179 - accuracy: 0.9016


[0.3179115653038025, 0.9016393423080444]

# Sample Prediction

In [79]:
sample = {
    "age": 60,
    "sex": 1,
    "cp": 1,
    "trestbps": 145,
    "chol": 233,
    "fbs": 1,
    "restecg": 2,
    "thalach": 150,
    "exang": 0,
    "oldpeak": 2.3,
    "slope": 3,
    "ca": 0,
    "thal": "fixed",
}

In [81]:
X_test_sample = vectorizer.transform(pd.DataFrame(sample, index=[0]))

In [82]:
model.predict(X_test_sample)

array([[0.40825787]], dtype=float32)